# 🐦 استخراج تغريدات نادي علم البيانات

هذا النوتبوك يستخدم مكتبة **snscrape** لجلب آخر 5 تغريدات من حساب نادي علم البيانات.

## المميزات:
- ✅ مجاني 100% - بدون API Keys
- ✅ بدون تسجيل - لا يحتاج حساب مطور
- ✅ سهل الاستخدام

In [1]:
# تثبيت مكتبة snscrape
!pip install snscrape

   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   -- ------------------------------------- 0.3/4.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.5/4.0 MB 1.4 MB/s eta 0:00:03
   ------- -------------------------------- 0.8/4.0 MB 1.3 MB/s eta 0:00:03
   ------------- -------------------------- 1.3/4.0 MB 1.5 MB/s eta 0:00:02
   --------------- ------------------------ 1.6/4.0 MB 1.6 MB/s eta 0:00:02
   ------------------ --------------------- 1.8/4.0 MB 1.3 MB/s eta 0:00:02
   ------------------ --------------------- 1.8/4.0 MB 1.3 MB/s eta 0:00:02
   -------------------- ------------------- 2.1/4.0 MB 1.3 MB/s eta 0:00:02
   ----------------------- ---------------- 2.4/4.0 MB 1.3 MB/s eta 0:00:02
   -------------------------- ------------- 2.6/4.0 MB 1.2 MB/s eta 0:00:02
   ---------------------------- ----------- 2.9/4.0 MB 1.2 MB/s eta 0:00:01
   ------------------------------

In [2]:
import snscrape.modules.twitter as sntwitter
from datetime import datetime

def get_last_5_tweets_free(username: str):
    """
    🆓 جلب آخر 5 تغريدات بدون API Keys باستخدام snscrape
    """
    print(f"🔍 جاري البحث عن تغريدات @{username}...")
    print("=" * 50)
    
    tweets_list = []
    
    try:
        # استخدام TwitterUserScraper لجلب تغريدات المستخدم
        scraper = sntwitter.TwitterUserScraper(username)
        
        for i, tweet in enumerate(scraper.get_items()):
            if i >= 5:  # نريد 5 تغريدات فقط
                break
            tweets_list.append(tweet)
        
        if not tweets_list:
            print(f"❌ لم يتم العثور على تغريدات لـ @{username}")
            return
        
        print(f"✅ تم العثور على {len(tweets_list)} تغريدات!\n")
        print(f"🐦 آخر 5 تغريدات لـ @{username}:\n")
        
        for i, tweet in enumerate(tweets_list, 1):
            print(f"📝 التغريدة {i}:")
            print(f"   📅 التاريخ: {tweet.date.strftime('%Y-%m-%d %H:%M')}")
            
            # عرض النص (مع اختصار إذا كان طويل)
            text = tweet.rawContent if hasattr(tweet, 'rawContent') else tweet.content
            if len(text) > 100:
                print(f"   💬 النص: {text[:100]}...")
            else:
                print(f"   💬 النص: {text}")
            
            print(f"   ❤️ إعجابات: {tweet.likeCount}")
            print(f"   🔄 ريتويت: {tweet.retweetCount}")
            print(f"   💬 ردود: {tweet.replyCount}")
            print(f"   🔗 الرابط: {tweet.url}")
            print("-" * 40)
        
        return tweets_list
        
    except Exception as e:
        print(f"❌ حدث خطأ: {e}")
        print("\n💡 نصائح:")
        print("   - تأكد من صحة اسم المستخدم")
        print("   - جرب تحديث المكتبة: pip install --upgrade snscrape")
        print("   - قد يكون الحساب خاص أو محظور")
        return None

In [5]:
# ==========================================
# 🚀 تشغيل الطريقة المجانية
# ==========================================

# جلب آخر 5 تغريدات لنادي علم البيانات
tweets = get_last_5_tweets_free("DSC_KAU")

🔍 جاري البحث عن تغريدات @DSC_KAU...


Error retrieving https://twitter.com/i/api/graphql/7jT5GT59P8IFjgxwqnEdQw/SearchTimeline?variables=%7B%22rawQuery%22%3A%22from%3ADSC_KAU%22%2C%22count%22%3A20%2C%22product%22%3A%22Latest%22%2C%22withDownvotePerspective%22%3Afalse%2C%22withReactionsMetadata%22%3Afalse%2C%22withReactionsPerspective%22%3Afalse%7D&features=%7B%22rweb_lists_timeline_redesign_enabled%22%3Afalse%2C%22blue_business_profile_image_shape_enabled%22%3Afalse%2C%22responsive_web_graphql_exclude_directive_enabled%22%3Atrue%2C%22verified_phone_label_enabled%22%3Afalse%2C%22creator_subscriptions_tweet_preview_api_enabled%22%3Afalse%2C%22responsive_web_graphql_timeline_navigation_enabled%22%3Atrue%2C%22responsive_web_graphql_skip_user_profile_image_extensions_enabled%22%3Afalse%2C%22tweetypie_unmention_optimization_enabled%22%3Atrue%2C%22vibe_api_enabled%22%3Atrue%2C%22responsive_web_edit_tweet_api_enabled%22%3Atrue%2C%22graphql_is_translatable_rweb_tweet_is_translatable_enabled%22%3Atrue%2C%22view_counts_everywhere_api

❌ حدث خطأ: 4 requests to https://twitter.com/i/api/graphql/7jT5GT59P8IFjgxwqnEdQw/SearchTimeline?variables=%7B%22rawQuery%22%3A%22from%3ADSC_KAU%22%2C%22count%22%3A20%2C%22product%22%3A%22Latest%22%2C%22withDownvotePerspective%22%3Afalse%2C%22withReactionsMetadata%22%3Afalse%2C%22withReactionsPerspective%22%3Afalse%7D&features=%7B%22rweb_lists_timeline_redesign_enabled%22%3Afalse%2C%22blue_business_profile_image_shape_enabled%22%3Afalse%2C%22responsive_web_graphql_exclude_directive_enabled%22%3Atrue%2C%22verified_phone_label_enabled%22%3Afalse%2C%22creator_subscriptions_tweet_preview_api_enabled%22%3Afalse%2C%22responsive_web_graphql_timeline_navigation_enabled%22%3Atrue%2C%22responsive_web_graphql_skip_user_profile_image_extensions_enabled%22%3Afalse%2C%22tweetypie_unmention_optimization_enabled%22%3Atrue%2C%22vibe_api_enabled%22%3Atrue%2C%22responsive_web_edit_tweet_api_enabled%22%3Atrue%2C%22graphql_is_translatable_rweb_tweet_is_translatable_enabled%22%3Atrue%2C%22view_counts_everyw

## 💾 حفظ التغريدات في ملف JSON (اختياري)

In [4]:
import json

def save_tweets_to_json(tweets, filename="dsc_tweets.json"):
    """
    حفظ التغريدات في ملف JSON
    """
    if not tweets:
        print("❌ لا توجد تغريدات لحفظها")
        return
    
    tweets_data = []
    for tweet in tweets:
        tweet_dict = {
            "id": tweet.id,
            "date": tweet.date.isoformat(),
            "content": tweet.rawContent if hasattr(tweet, 'rawContent') else tweet.content,
            "likes": tweet.likeCount,
            "retweets": tweet.retweetCount,
            "replies": tweet.replyCount,
            "url": tweet.url
        }
        tweets_data.append(tweet_dict)
    
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(tweets_data, f, ensure_ascii=False, indent=2)
    
    print(f"✅ تم حفظ {len(tweets_data)} تغريدات في: {filename}")

# حفظ التغريدات (إذا تم جلبها بنجاح)
if tweets:
    save_tweets_to_json(tweets)